In [2]:
#imports
from keras.initializers import glorot_uniform
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [3]:
#generate data
x_train = np.random.random((20,8)) 
y_train = np.random.randint(2, size = (20, 1))
x_test = np.random.random((20,8))
y_test = np.random.randint(2, size=(20,1))

In [4]:
#model
model = models.Sequential()
model.add(layers.Dense(16, kernel_initializer = 'random_uniform', input_dim = 8, activation = 'sigmoid'))
model.add(layers.Dense(32, activation = 'sigmoid'))
model.add(layers.Dense(1,activation='sigmoid'))
#compile
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metric = ['accuracy'])
#find fitness
fitness = model.evaluate(x_test, y_test, batch_size=128, verbose = 0)
print(fitness)

1.2192671298980713


In [5]:
#Work Functions
def getWeights(particle):
    model.set_weights(particle)
    #get weights
    d16w, d16b = model.layers[0].get_weights()
    d32w, d32b = model.layers[1].get_weights()
    d1w, d1b = model.layers[2].get_weights()
    return d16w, d32w, d1w

def setWeights(n, d16w, d32w, d1w, particle, after):
    a, d16b = model.layers[0].get_weights()
    b, d32b = model.layers[1].get_weights()
    c, d1b = model.layers[2].get_weights()
    model.layers[0].set_weights([d16w,d16b])
    model.layers[1].set_weights([d32w, d32b])
    model.layers[2].set_weights([d1w, d1b])
    newWeights = model.get_weights()
    population[n][0] = newWeights
    population[n][1] = after

def buildPopulation(numParticles):
    population = []
    policies = []
    for x in range(numParticles):
        initial_weights = model.get_weights()
        k_eval = lambda placeholder: placeholder.numpy()
        new_weights = [k_eval(glorot_uniform()(w.shape)) for w in initial_weights]
        model.set_weights(new_weights)
        #find fitness
        fitness = model.evaluate(x_test, y_test, batch_size=128, verbose = 0)
        population.append([new_weights, fitness, policies])
    return population

def addPopulation(population, numParticles):
    policies = []
    for x in range(numParticles):
        initial_weights = model.get_weights()
        k_eval = lambda placeholder: placeholder.numpy()
        new_weights = [k_eval(glorot_uniform()(w.shape)) for w in initial_weights]
        model.set_weights(new_weights)
        #find fitness
        fitness = model.evaluate(x_test, y_test, batch_size=128, verbose = 0)
        population.append([new_weights, fitness, policies])
    return population    

def findPolicies(population):
    policies = []
    for x in population:
        if (x[1] > 0.80):
            print("create policy")
            
##Policy Data
# Policy Format
# LayerNum, yStart, yStop, ySkip, xStart, xStop, xSkip, increment 
#
# Layers: (8, 16) (16, 32) (32, 1)
def createPolicy(layer,yS, yE, yI, xS, xE, xI, inc):
    if (layer == "input"):
        policy = ("input", yS, yE, yI, xS, xE, xI, inc)
    elif (layer == "hidden"):
        policy = ("hidden", yS, yE, yI, xS, xE, xI, inc)
    elif (layer == "output"):
        policy = ("output", yS, yE, yI, xS, xE, xI, inc)
    return policy

def buildPolicy(startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc):
    #8,16
    policy1 = ("input", startY, endY, skipY, startX, endX, skipX, inc)
    #16,32
    policy2 = ("hidden", startX, endX, inc, startZ, endZ, skipZ, inc)
    #32,1
    policy3 = ("output", startZ, endZ, inc, 0, 1, skipZ, inc)
    return policy

##An idea I was playing with until it started taking, too, long##
def strong(startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist):
    persist = true
    return startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist

def charasmatic(startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist):
    #take a random trait from the populus
    return startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist

def dexterous(startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist):
    #find the best direction
    return startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist

def agile(startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist):
    #travels quickly in a direction
    skipY = 2
    skipX = 2
    skipZ = 2
    inc *= 2
    return startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist

def intelligent(startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist):
    #copy the best trait amongst the populus
    return startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist

def wise(startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist):
    #copy the best last trait amongst the populus
    return startY, endY, skipY, startX, endX, skipX, startZ, endZ, skipZ, inc, persist

def buildTraits():
    traits = []
    traits.append("Strong", buildPolicy(a,b,c,d,e,f,g,h,i))
############

#for each: Entity, Accuracy, Policy
#Policy  : LayerNum, yStart, yStop, ySkip, xStart, xStop, xSkip, increment
def applyPolicy(policy, population):
    improvements = 0
    numPop = 0
    totAmount = 0
    if (policy[0] == "input"):
        for n in population:
            x, y, z =  getWeights(n[0])
            beforeChange = x + 0
            x[policy[1]:policy[2]:policy[3],policy[4]:policy[5]:policy[6]] += policy[7]
            x[x>1] = 1
            x[x<-1] = -1
            improved, amount, after = checkImprovement(beforeChange,x,y,z, "input")
            if (improved):
                improvements += 1
                totAmount += amount
                setWeights(numPop,x,y,z,population, after)
                print(numPop, " imp to ", after)
            numPop += 1
    elif (policy[0] == "hidden"):
        for n in population:
            x, y, z =  getWeights(n[0])
            beforeChange = y + 0
            y[policy[1]:policy[2]:policy[3],policy[4]:policy[5]:policy[6]] += policy[7]
            y[y>1] = 1
            y[y<-1] = -1
            improved, amount, after = checkImprovement(beforeChange,x,y,z, "hidden")
            if (improved):
                improvements += 1
                totAmount += amount
                setWeights(numPop,x,y,z,population, after)
                print(numPop," imp to ", after)
            numPop += 1
    elif (policy[0] == "output"):
        for n in population:
            x, y, z =  getWeights(n[0])
            beforeChange = z + 0
            z[policy[1]:policy[2]:policy[3],policy[4]:policy[5]:policy[6]] += policy[7]
            z[z>1] = 1
            z[z<-1] = -1
            improved, amount, after = checkImprovement(beforeChange,x,y,z, "output")
            if (improved):
                improvements += 1
                totAmount += amount
                setWeights(numPop,x,y,z,population, after)
                print(numPop, " imp to ", after)
            numPop += 1
    return improvements/numPop, totAmount
        
def checkImprovement(beforeChange, x, y, z, layer):
    w1, b1 = model.layers[0].get_weights()
    w2, b2 = model.layers[1].get_weights()
    w3, b3 = model.layers[2].get_weights()
    
    #After
    model.layers[0].set_weights([x,b1])
    model.layers[1].set_weights([y,b2])
    model.layers[2].set_weights([z,b3])
    after = model.evaluate(x_test, y_test, batch_size=128, verbose = 0)
    #print(after)
    
    #Before
    if(layer == "input"):
        model.layers[0].set_weights([beforeChange,b1])
    elif(layer == "hidden"):
        model.layers[1].set_weights([beforeChange,b2])
    elif(layer == "output"):
        model.layers[2].set_weights([beforeChange,b3])
    before = model.evaluate(x_test, y_test, batch_size=128, verbose = 0)
    #print(before)

    return after > before, after - before, after

In [6]:
#
population = buildPopulation(3)
population = addPopulation(population, 2)

In [9]:
printAccuracies(population)

########## Entity 0  Accuracy:  0.705622673034668 ##########
########## Entity 1  Accuracy:  1.7593977451324463 ##########
########## Entity 2  Accuracy:  0.8744238018989563 ##########
########## Entity 3  Accuracy:  0.9605425596237183 ##########
########## Entity 4  Accuracy:  1.1074048280715942 ##########


In [10]:
x, y, z = getWeights(population[0][0])
print(x.shape, y.shape, z.shape)

(8, 16) (16, 32) (32, 1)


In [11]:
findPolicies(population)

create policy
create policy
create policy
create policy


In [12]:
#Creating policy set
#Layers: (8, 16) (16, 32) (32, 1)
policies = []
policies.append(createPolicy("input", 0, 3, 1, 2, 5, 1, 0.1))
policies.append(createPolicy("hidden", 0, 3, 1, 2, 5, 1, 0.1))
policies.append(createPolicy("output", 0, 18, 1, 0, 1, 1, 0.1))
policies.append(createPolicy("input", 3, 9, 1, 7, 12, 1, 0.1))
policies.append(createPolicy("hidden", 4, 14, 2, 0, 15, 3, 0.1))
policies.append(createPolicy("output", 4, 12, 2, 0, 1, 1, 0.1))

In [20]:
policies[0] = createPolicy("hidden", 0, 15, 3, 0, 15, 1, -0.01)

In [21]:
policies[1] = createPolicy("hidden", 0, 15, 2, 16, 31, 3, -0.1)

In [22]:
policies[5] = createPolicy("input", 0, 7, 3, 0, 15, 2, 0.15)

In [23]:
for n in policies:
    print("policy", n)
    print("Population: 5",applyPolicy(n, population))

policy ('hidden', 0, 15, 3, 0, 15, 1, -0.01)
Population: 5 (0.0, 0)
policy ('hidden', 0, 15, 2, 16, 31, 3, -0.1)
4  imp to  1.416395664215088
Population: 5 (0.2, 0.023497939109802246)
policy ('output', 0, 18, 2, 0, 1, 1, -0.1)
0  imp to  1.0627262592315674
1  imp to  2.320910692214966
2  imp to  1.2186930179595947
3  imp to  1.4819858074188232
4  imp to  1.6960504055023193
Population: 5 (1.0, 1.1927917003631592)
policy ('input', 3, 9, 1, 7, 12, 1, 0.1)
0  imp to  1.0667250156402588
1  imp to  2.325803518295288
2  imp to  1.2291746139526367
3  imp to  1.4844167232513428
Population: 5 (0.8, 0.021804094314575195)
policy ('hidden', 4, 14, 2, 0, 15, 3, 0.1)
0  imp to  1.093242883682251
2  imp to  1.2465147972106934
3  imp to  1.541395664215088
4  imp to  1.7307615280151367
Population: 5 (0.8, 0.13554811477661133)
policy ('input', 0, 7, 3, 0, 15, 2, 0.15)
0  imp to  1.0965790748596191
1  imp to  2.3330881595611572
2  imp to  1.2495111227035522
3  imp to  1.5486396551132202
4  imp to  1.74618

In [24]:
printAccuracies(population)

########## Entity 0  Accuracy:  1.0965790748596191 ##########
########## Entity 1  Accuracy:  2.3330881595611572 ##########
########## Entity 2  Accuracy:  1.2495111227035522 ##########
########## Entity 3  Accuracy:  1.5486396551132202 ##########
########## Entity 4  Accuracy:  1.7461868524551392 ##########


In [219]:
printAccuracies(population)

########## Entity 0  Accuracy:  0.6968516111373901 ##########
########## Entity 1  Accuracy:  1.4269683361053467 ##########
########## Entity 2  Accuracy:  0.7396165132522583 ##########
########## Entity 3  Accuracy:  0.8059670329093933 ##########
########## Entity 4  Accuracy:  0.8139412999153137 ##########


In [222]:
printAccuracies(population)

########## Entity 0  Accuracy:  0.7758603096008301 ##########
########## Entity 1  Accuracy:  1.6688785552978516 ##########
########## Entity 2  Accuracy:  0.8469123840332031 ##########
########## Entity 3  Accuracy:  0.9204739332199097 ##########
########## Entity 4  Accuracy:  0.8139445185661316 ##########


In [51]:
printPopulation(population)

########## Entity 0  Accuracy:  0.6787484884262085 ##########
INPUT LAYER
 [[ 0.15074539 -0.03554606 -0.17096198 -0.4002521   0.36787903  0.05870104
   0.0712204   0.34592783 -0.44696426  0.06325543  0.48108125  0.37429285
   0.2609024   0.47629547  0.2008977   0.14639556]
 [ 0.26432836 -0.29748595 -0.4958005  -0.4740908  -0.29293406  0.49450564
   0.28132582  0.12826931 -0.12684178 -0.19379127 -0.38749254  0.43785906
  -0.41776013 -0.11295485  0.05966806 -0.04230094]
 [-0.48975265  0.07610285  0.22173905  0.05521631 -0.1374749  -0.36309946
   0.41788232 -0.29002452  0.19332695  0.07848883  0.01442349  0.03828752
  -0.15009713  0.20353794 -0.23316205  0.17609215]
 [ 0.4417585   0.10748255 -0.25116658 -0.41352892 -0.4030546  -0.4280244
   0.4926722  -0.26057017 -0.18395245  0.19820142 -0.38794267 -0.31279814
  -0.15791059  0.31897354  0.0364846   0.1789316 ]
 [ 0.0226717   0.36503518 -0.22301579  0.19865763  0.0802958   0.15807486
   0.38768363  0.43325245 -0.49932396  0.4273367  -0.021

In [8]:
#Print Functions
def printPopulation(population):
    i = 0
    for n in population:
        print("########## Entity", i, " Accuracy: ", n[1],"##########")
        printEntity(n[0])
        printPolicies(n[2])
        i += 1
        
def printEntity(entity):
    x, y, z = getWeights(entity)
    print("INPUT LAYER\n", x, "\n\n", "HIDDEN 1\n", y, "\n\n", "OUTPUT LAYER\n", z, "\n\n")
    
def printPolicies(policies):
    print("<<POLICIES>>")
    for n in policies:
        print(n)
    print("\n")
    
def printAccuracies(population):
    i = 0
    for n in population:
        print("########## Entity", i, " Accuracy: ", n[1],"##########")
        i += 1